In [ ]:
# Generate all comments and reactions.

from github import Github as pg
#Error handling -- retries and timeout isnt handled by Github Exception handling
from github import RateLimitExceededException, BadCredentialsException, BadAttributeException, GithubException, UnknownObjectException, BadUserAgentException
import requests #retries and timeout Github Exception handling
import time #sleep the process for sometime

import csv 
from collections.abc import Iterable #To check if a datatype is iterable 
from pandas import *

MY_TOTAL_TOKENS = 8
access_token1 = <add_token_here> 
access_token2 = <add_token_here> 
access_token3 = <add_token_here> 
access_token4 = <add_token_here> 

access_token5 = <add_token_here> 
access_token6 = <add_token_here> 
access_token7 = <add_token_here> 
access_token8 = <add_token_here> 

# Set initial API token
api_token_queue = [access_token1,access_token2,access_token3,access_token4,access_token5,access_token6,access_token7,access_token8]
languages = [ "HCL", "TypeScript", "Rust", "Lua", "Go", "Shell", "Makefile", "C", "Kotlin", "Python"]

In [ ]:
#Read all the project names
#fetch all issues details to local csv
token_i = 0
g = pg(api_token_queue[token_i], per_page=100, retry=5)
# Create a CSV file and write the headers 
for lang in languages:
    # reading CSV file
    print("Hey there, I'm working on language:",lang)
    data = read_csv("../Output/"+lang+".csv") 
    # converting column data to list
    repo_names = data['Repo Name'].tolist()
    with open("../Output/Issues/"+lang+".csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Number", "Title", "Body", "State", "Created At", "Updated At", "Comments URL", "#Comments","#Reactions", "Comments", "Reactions"])
        for repo_name in repo_names:
            this_rate= str(g.get_rate_limit())
            this_rate= int(this_rate.split(",")[1].split("=")[1])
            if(this_rate < 1000):
                token_i =(token_i + 1) % MY_TOTAL_TOKENS
            while True:
                try:
                    g = pg(api_token_queue[token_i], per_page=100, retry=5)
                    print(f"Extracting issues from {repo_name} repo using token #{token_i}")
                    repo = g.get_repo(repo_name)
                    # Fetch all issues (both open and closed)
                    issues = repo.get_issues(state='all', sort='created', direction='asc')
#                     only_issues = [issue for issue in issues if issue.pull_request is None]
                    only_issues = issues
                    # Write the issue data to the CSV file
                    for issue in only_issues:
                        while True:
                            try:
                                # extracting comments(inc reactions) from the issues
                                issues_comments = issue.get_comments()
                                cmnt_cnt = 0 #To store the number of comments done on this issue
                                rxn_cnt = 0 #To store teh number of reactions on the issues
                                issue_comments = []
                                issue_reactions = []
                                for comment in issues_comments:
                                    issue_comments.append(comment.body)
                                    cmnt_cnt +=1
                                    this_cmnt_rxns = []
                                    any_rxn = 0 # To check if there are any reactions on this comment
                                    for reaction in comment.get_reactions():
                                        #if no reactions, then just append null or put comma and proceed.
                                        if reaction.content == []:
                                            this_cmnt_rxns.append('NA')
                                        else:
                                            this_cmnt_rxns.append(reaction.content)
                                            any_rxn +=1
                                            rxn_cnt +=1
                                    #if no reactions, then just append null or put comma and proceed.
                                    if any_rxn > 0:
                                        issue_reactions.append(this_cmnt_rxns)
                                    else: 
                                        issue_reactions.append('NA')
                                #Write row into the csv
                                writer.writerow([issue.number, issue.title, issue.body, issue.state, issue.created_at, issue.updated_at, issue.comments_url, cmnt_cnt, rxn_cnt, issue_comments, issue_reactions])
                            except RateLimitExceededException as e:
                                print(e.status)
                                print('Inner Loop: Rate Limit exceeded')
                                time.sleep(300) #5 minutes
                                continue
                            except BadCredentialsException as e:
                                print(e.status)
                                print("Inner Loop: Bad Credentials exception")
                                break
                            except UnknownObjectException as e:
                                print(e.status)
                                print("The repository doesn't exist anymore")
                                break
                            except GithubException as e:
                                print(e.status)
                                print("Inner Loop: Github General Exception")
                                break
                            except requests.exceptions.ConnectionError as e:
                                print("Inner Loop: Retries limit exceeded")
                                print(str(e))
                                time.sleep(10)
                                continue
                            except requests.exceptions.Timeout as e:
                                print("Inner Loop: Timeout exception")
                                print(str(e))
                                time.sleep(10)
                                continue
                            break
                except RateLimitExceededException as e:
                    print(e.status)
                    print('Rate Limit exceeded')
                    time.sleep(300) #5 minutes
                    continue
                except BadCredentialsException as e:
                    print(e.status)
                    print("Bad Credentials exception")
                    break
                except UnknownObjectException as e:
                    print(e.status)
                    print("The repository doesn't exist anymore")
                    break
                except GithubException as e:
                    print(e.status)
                    print("Github General Exception")
                    break
                except requests.exceptions.ConnectionError as e:
                    print("Retries limit exceeded")
                    print(str(e))
                    time.sleep(10)
                    continue
                except requests.exceptions.Timeout as e:
                    print("Timeout exception")
                    print(str(e))
                    time.sleep(10)
                    continue
                break
    print("Hey there, I'm done with language:",lang)